In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
##load trained model, scaler, one hot encoder
model = load_model('model.h5')

##load encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file: 
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [6]:
##example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France', #think how to convert france to numerical value
    'Gender': 'Female', #think how to convert female to numerical value
    'Age': 40,      
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000 #at last scale down all the numerical value using std scaler
}

In [8]:
##onehot encoding Geography
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\HP\Desktop\ANN-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,40,3,60000,2,1,1,50000


In [10]:
##encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [ ]:
## concat one hot encoded columns with input df
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
##scaling i/p data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [15]:
##predict churn
prediction =model.predict(input_scaled)
prediction


1/1 [==============================] - 0s 189ms/step


array([[0.0374186]], dtype=float32)

In [16]:
prediction_prob = prediction[0][0]
prediction_prob

0.037418596

In [17]:
if prediction_prob > 0.5:
    print(f'The customer is likely to churn with a probability of {prediction_prob:.2f}')
else:
    print(f'The customer is unlikely to churn with a probability of {prediction_prob:.2f}')

The customer is unlikely to churn with a probability of 0.04
